## DIA 017: Optimización Avanzada del Fine-Tuning y Monitorización Mejorada

Optimizar el Fine-Tuning Avanzado del Modelo:

Descongelar Más Capas del Modelo Base (VGG16): Permitir que más capas aprendan características específicas del dataset MNIST.
Aplicar Técnicas de Regularización Adicionales: Como Batch Normalization para estabilizar y acelerar el entrenamiento.
Implementar un Scheduler de Tasa de Aprendizaje: Ajustar dinámicamente la tasa de aprendizaje durante el entrenamiento para mejorar la convergencia.
Mejorar la Monitorización del Entrenamiento:

Integrar TensorBoard: Para visualizar métricas de entrenamiento en tiempo real.
Guardar Modelos en Diferentes Puntos: Además del mejor modelo, guardar checkpoints a intervalos regulares.

In [2]:
# 1. Importación de Librerías
# ------------------------------------
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import datetime  # Para timestamp en TensorBoard


In [ ]:
# 2. Cargar el Modelo Final Entrenado
# ------------------------------------

# Cargar el modelo completamente entrenado del Día 16
modelo_final = tf.keras.models.load_model('modelo_cnn_transfer_learning_mnist_final.h5')
print("Modelo final cargado exitosamente.")


In [ ]:
# 3. Descongelar Más Capas para Fine-Tuning Avanzado
# ------------------------------------

# Obtener la capa base del modelo (VGG16)
base_model = modelo_final.layers[0]

# Mostrar las capas actuales y su estado de entrenabilidad
print("Capas antes de descongelar:")
for i, layer in enumerate(base_model.layers):
    print(f"{i}: {layer.name} - {'Entrenable' if layer.trainable else 'Congelada'}")

# Descongelar todas las capas a partir de 'block4_conv1'
set_trainable = False
for layer in base_model.layers:
    if layer.name == 'block4_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# Verificar el estado de las capas después de descongelar
print("\nCapas después de descongelar:")
for i, layer in enumerate(base_model.layers):
    print(f"{i}: {layer.name} - {'Entrenable' if layer.trainable else 'Congelada'}")


In [ ]:
# 4. Añadir Capas de Regularización Adicionales
# ------------------------------------

# Reconstruir el modelo con Batch Normalization
modelo_optimizado = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),  # Añadir Batch Normalization
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')  # 10 clases para MNIST
])

# Resumen del modelo optimizado
modelo_optimizado.summary()


In [ ]:
# 5. Recompilar el Modelo con un Scheduler de Tasa de Aprendizaje
# ------------------------------------

# Recompilar el modelo optimizado con una tasa de aprendizaje más baja
modelo_optimizado.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])


In [ ]:
# 6. Configurar TensorBoard para Monitorización Mejorada
# ------------------------------------

# Crear un timestamp para los logs de TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Definir callbacks adicionales
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=1e-7, verbose=1)


In [ ]:
# 7. Reentrenar el Modelo con Fine-Tuning Optimizado
# ------------------------------------

# Definir callbacks para el entrenamiento
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('mejor_modelo_ft.h5', monitor='val_loss', save_best_only=True, verbose=1),
    reduce_lr,
    tensorboard_callback
]

# Reentrenar el modelo optimizado con fine-tuning avanzado
history_ft = modelo_optimizado.fit(
    train_ds,
    epochs=10,  # Puedes ajustar el número de épocas según sea necesario
    validation_data=val_ds,
    callbacks=callbacks
)


In [ ]:
# 8. Evaluar y Visualizar el Rendimiento Mejorado
# ------------------------------------

# Evaluar el modelo después del fine-tuning optimizado
test_loss_ft, test_acc_ft = modelo_optimizado.evaluate(val_ds, verbose=2)
print(f'\nPrecisión en el conjunto de prueba después del fine-tuning optimizado: {test_acc_ft:.4f}')

# Graficar precisión y pérdida durante el fine-tuning optimizado
plt.figure(figsize=(12, 4))

# Precisión
plt.subplot(1, 2, 1)
plt.plot(history_ft.history['accuracy'], label='Entrenamiento')
plt.plot(history_ft.history['val_accuracy'], label='Validación')
plt.title('Precisión durante el Fine-Tuning Optimizado')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()

# Pérdida
plt.subplot(1, 2, 2)
plt.plot(history_ft.history['loss'], label='Entrenamiento')
plt.plot(history_ft.history['val_loss'], label='Validación')
plt.title('Pérdida durante el Fine-Tuning Optimizado')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()

plt.show()


In [ ]:
# 9. Guardar el Modelo Final Optimizado
# ------------------------------------

# Guardar el modelo completamente optimizado
modelo_optimizado.save('modelo_cnn_transfer_learning_mnist_final_optimizado.h5')
print("Modelo final optimizado guardado como 'modelo_cnn_transfer_learning_mnist_final_optimizado.h5'")


In [ ]:
# 10. Hacer Predicciones con el Modelo Final Optimizado
# ------------------------------------

# Hacer predicciones sobre el conjunto de prueba con el modelo optimizado
predicciones_ft = modelo_optimizado.predict(val_ds)
clases_predichas_ft = np.argmax(predicciones_ft, axis=1)

# Mostrar predicción para la primera imagen del conjunto de prueba
indice = 0
print(f"Etiqueta real: {y_test[indice]}")
print(f"Predicción: {clases_predichas_ft[indice]}")
print(f"Probabilidades: {predicciones_ft[indice]}")

# Visualizar la imagen con su predicción
def mostrar_prediccion(matriz, etiquetas, predicciones, indice):
    plt.figure(figsize=(2,2))
    plt.imshow(matriz[indice].reshape(28, 28), cmap='gray')
    plt.title(f"Real: {etiquetas[indice]}\nPred: {predicciones[indice]}")
    plt.axis('off')
    plt.show()

# Mostrar la primera imagen del conjunto de prueba con su predicción
mostrar_prediccion(X_test, y_test, clases_predichas_ft, indice)


In [ ]:
# 11. Evaluación Más Detallada (Reporte de Clasificación y Matriz de Confusión)
# ------------------------------------

# Imprimir el reporte de clasificación
print("Reporte de Clasificación después del Fine-Tuning Optimizado:\n")
print(classification_report(y_test, clases_predichas_ft))

# Generar la matriz de confusión
cm_ft = confusion_matrix(y_test, clases_predichas_ft)

# Visualizar la matriz de confusión
plt.figure(figsize=(10,8))
sns.heatmap(cm_ft, annot=True, fmt='d', cmap='Greens', 
            xticklabels=np.unique(y_train), 
            yticklabels=np.unique(y_train))
plt.xlabel('Predicción')
plt.ylabel('Realidad')
plt.title('Matriz de Confusión después del Fine-Tuning Optimizado')
plt.show()